In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
import os

def solve_vec(method, f, y0, rng, h):
    t0, tn = rng
    t = np.arange(t0,tn,h)
    y = np.zeros((len(t), )+y0.shape, np.float64)
    y[0] = y0
    
    for ti in range(1, len(t)):
        y[ti] = method(f, t[ti-1], y[ti-1], h)
    return t, y

def euler_method(f, t, y, h):
    return y + h * f(t, y)

def midpoint_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    return y + h * K2

def RK4_method(f, t, y, h):
    K1 = f(t, y)
    K2 = f(t+h*0.5, y+(h*0.5*K1))
    K3 = f(t+h*0.5, y+(h*0.5*K2))
    K4 = f(t+h, y+(h*K3))
    return y + h*(K1/6+K2/3+K3/3+K4/6)

In [ ]:
def render_frame(y, y_prev, l, canvas):
    shape = canvas.shape
    pos_last = (shape[0]//2, shape[0]//2)
    pos_prev_last = pos_last
    canvas[:] = canvas*[0.99,0.99,0.99]
    for i in range(len(y)):
        theta = y[i, 0]
        theta_prev = y_prev[i, 0]
        pos = (pos_last[0]+int(l[i]*np.sin(theta)), pos_last[1]+int(l[i]*np.cos(theta)))
        pos_prev = (pos_prev_last[0]+int(l[i]*np.sin(theta_prev)), pos_prev_last[1]+int(l[i]*np.cos(theta_prev)))
        cv2.line(canvas, pos_prev, pos, [int(180-180*(float(i)/len(y))),64,int(180*(float(i)/len(y)))], 2)
        pos_last = pos
        pos_prev_last = pos_prev
    
    img = canvas.copy()

    # spring
    pos_last = (shape[0]//2, shape[0]//2)
    for i in range(len(y)):
        theta = y[i, 0]
        pos = (pos_last[0]+int(l[i]*np.sin(theta)), pos_last[1]+int(l[i]*np.cos(theta)))
        cv2.line(img, pos_last, pos, [64,64,64], 2)        
        pos_last = pos

    pos_last = (shape[0]//2, shape[0]//2)
    for i in range(len(y)):
        theta = y[i, 0]
        pos = (pos_last[0]+int(l[i]*np.sin(theta)), pos_last[1]+int(l[i]*np.cos(theta)))    
        r = 12
        cv2.circle(img, pos, r, [int(255-255*(float(i)/len(y))),32,int(255*(float(i)/len(y)))], -1)
        cv2.circle(img, (pos[0]+r//3,pos[1]-r//3), r//3, [235,240,245], -1)
        pos_last = pos
        
    return img

def balls_animation(t, y, l, size, pre=''):
    path = 'output/'+pre+'_'+str(time.time())+'.avi'
    if(int(cv2.__version__.split('.')[0]) < 3):
        fourcc = cv2.cv.CV_FOURCC(*'MJPG')
    else:
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    video = cv2.VideoWriter(path, fourcc, len(t)//t[-1], (size,size))
    
    canvas = np.zeros((size,size,3))
    
    l_scaled = l * ((0.4 * size)/np.sum(l))
    for i in range(len(t)):
        if i>0:
            y_prev = y[i-1]
        else:
            y_prev = y[i]
        frame = render_frame(y[i], y_prev, l_scaled, canvas)
        video.write(frame.astype(np.uint8))
    video.release()
    
    path_mp4 = path[:-4]+'.mp4'
    os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{path_mp4}'".format(input = path, path_mp4=path_mp4))
    os.remove(path)
    return path_mp4

In [ ]:
from sympy import diff, symbols, cos, sin, Derivative, solve, Symbol

def euler_lagrange(lagrangian, t, x_dx_ddx):
    equations = []
    for syms in x_dx_ddx:
        x,dx,ddx = syms
        dlag_x = diff(lagrangian, x)
        dlag_dx = diff(lagrangian, dx)
        for syms2 in x_dx_ddx:
            x2,dx2,ddx2 = syms2
            dlag_dx = dlag_dx.subs([(dx2, Derivative(x2(t), t)), (x2, x2(t))])
        dlag_dx_t = diff(dlag_dx, t)
        for syms2 in x_dx_ddx:
            x2,dx2,ddx2 = syms2
            dlag_dx_t = dlag_dx_t.subs([
                    (Derivative(x2(t), t, t), ddx2),
                    (Derivative(x2(t), t), dx2),
                    (x2(t), x2)])
        equations.append(dlag_dx_t-dlag_x)
    return equations

In [ ]:
N = 3
t = symbols('t')
syms = [ symbols('th[%d] dth[%d] ddth[%d]'%(i,i,i)) for i in range(N) ]

lag = 0
h = 0
velocity = [0, 0]
for i in range(N):
    th, dth, ddth = syms[i]
    # E_potential
    h += (Symbol('l[%d]'%(i,))-Symbol('l[%d]'%(i,))*cos(th))
    lag -= Symbol('m[%d]'%(i,))*Symbol('g')*h
    # E_kinetic
    velocity[0] += Symbol('l[%d]'%(i,))*dth*sin(th+np.pi/2)
    velocity[1] += Symbol('l[%d]'%(i,))*dth*cos(th+np.pi/2)
    # lagrangian
    lag += 0.5*Symbol('m[%d]'%(i,))*(velocity[0]**2+velocity[1]**2)
    #lag += 0.5*m[i]*(dth*l[i])**2

equations = euler_lagrange(lag,t,syms)
ddxs = [ syms[i][2] for i in range(N) ]
functions = solve(equations, ddxs)

In [ ]:
with open('functions%d.py'%(N,), 'w') as fd:
    fd.write('from numpy import sin, cos, zeros_like\n')
    fd.write('def force(t, y):\n')
    fd.write('    values = zeros_like(y)\n')
    f = functions
    for i in range(N):
        th, dth, ddth = syms[i]
        for j in range(N):
            _, _, ddthj = syms[j]
            f[ddthj] = f[ddthj].subs([(th,Symbol('y[%d,0]'%(i,))),(dth,Symbol('y[%d,1]'%(i,)))])
    for i in range(N):
        _, _, ddthi = syms[i]
        fd.write('    values[%d,0] = y[%d,1]\n'%(i,i))
        fd.write('    values[%d,1] = '%(i,)+str(f[ddthi])+'\n')
    fd.write('    return values\n')

In [ ]:
import os
filelist = [ f for f in os.listdir(".") if f.endswith(".pyc") ]
for f in filelist:
    os.remove(f)

from functions3 import force as f

g = 9.80665
#l = np.ones(N)
l = np.array([1,1,2])
m = np.ones(N)

# Initial state
y0 = np.array([ [1.57,0] for i in range(N) ], np.float64)

rng = (0,30)
h = 1/30.
t, y = solve_vec(RK4_method, f, y0, rng, h)

In [ ]:
path = balls_animation(t, y, l, 800, pre='t6')

HTML('<video width="800" height="800" controls><source src="'+path+'" type="video/mp4"></video>')